<a href="https://colab.research.google.com/github/juan-villa02/medical_vqa_vlm/blob/main/code/notebooks/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VQA Model - BERT + ResNet18

## Libraries/Dependencies

In [1]:
# PyTorch framework
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
# Pillow (image handling)
from PIL import Image
# Transformers (Hugginface)
from transformers import BertModel, BertTokenizer
# Matplotlib
import matplotlib.pyplot as plt
# Numpy
import numpy as np
# Extra dependencies
import json
import os
from tqdm import tqdm

In [2]:
# Set training device to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Paths

In [ ]:
#from google.colab import drive
#from shutil import copyfile
#drive.mount('/content/drive')

In [ ]:
path_dir = '.' #path to drive folder '/content/drive/MyDrive/TFG-VQA/data/bases de datos.zip'

ISIC (Fusion)

Images path: drive/content/TFG Juan Villanueva/bases de datos/ISIC_2016/images

Test -> ISBI2016_ISIC_Part1_Test_Data_orig
Train -> ISBI2016_ISIC_Part3_Training_Data_orig


Pizarro (Melanoma)

Images path: drive/content/bases de datos/pizarro/images

500 imágenes

Las 46 primeras son más complejas, en cuanto a diagnóstico. (Entrega 1)

El resto están más equilibradas (Entrega 2-5)


## VQA Dataset - Images & QA

In [ ]:
# Dataset class to load images and questions
class VQADataset(Dataset):
    def __init__(self, data_dir, json_file, tokenizer, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.tokenizer = tokenizer

        with open(json_file, 'r') as f:
            self.data = json.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image_path = os.path.join(self.data_dir, item['image_id'])
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        question = item['question']
        tokenized_question = self.tokenizer(question, return_tensors='pt', padding=True, truncation=True)

        return image, tokenized_question, item['qa_pairs'][0]['answer']